In [1]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from random import sample

Using TensorFlow backend.


### Load Vocab

In [2]:
vocab = {}
idToWord = {}
with open('vocab.txt') as f:
    words = f.read().splitlines()
    for wordIndex in words:
        word, index = wordIndex.split(' -----> ')
        vocab[word] = index
        idToWord[int(index)] = word

In [4]:
vocab['<pad_token>']

'1001'

### Create input squences

In [13]:
look_back_len = 250 + 1
sequences = []
vocabulary_size = len(vocab)

with open('int-seq.txt') as f:
    files = f.read().splitlines()
    for file in files:
        numbers = list(map(int, file.split(',')[:-1]))
        for i in range(look_back_len, len(numbers)):
            seq = numbers[i-look_back_len:i]
            sequences.append(seq)
    sequence_small = sample(sequences, int(len(sequences)*0.025))

    n_sequences = np.empty([len(sequence_small), look_back_len], dtype='int32')
    for i in range(len(sequence_small)):
        n_sequences[i] = sequences[i]

print(n_sequences.shape)

(196667, 251)


### Convert output to one hot encoded vector

In [14]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
print(len(train_targets))

train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_targets[0])

196667
[0. 1. 0. ... 0. 0. 0.]


### Train Model

In [15]:
import keras
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save("model_250Window{}.hd5".format(epoch))

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

saver = CustomSaver()

model.fit(train_inputs,train_targets, epochs = 20 ,verbose=1, validation_split=0.3, callbacks=[saver])
model.save("mymodel-250Window.h5")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 250)          250500    
_________________________________________________________________
lstm_5 (LSTM)                (None, 250, 50)           60200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_6 (Dense)              (None, 1002)              51102     
Total params: 384,552
Trainable params: 384,552
Non-trainable params: 0
_________________________________________________________________
None


c:\Miniconda\envs\azureml_py36_tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 137666 samples, validate on 59001 samples
Epoch 1/20
137666/137666 [==============================] - 2063s 15ms/step - loss: 2.3732 - accuracy: 0.4626 - val_loss: 2.2593 - val_accuracy: 0.5359
Epoch 2/20
137666/137666 [==============================] - 2053s 15ms/step - loss: 1.5392 - accuracy: 0.6397 - val_loss: 2.1857 - val_accuracy: 0.5579
Epoch 3/20
137666/137666 [==============================] - 2053s 15ms/step - loss: 1.2958 - accuracy: 0.6810 - val_loss: 2.1570 - val_accuracy: 0.5673
Epoch 4/20
137666/137666 [==============================] - 2047s 15ms/step - loss: 1.1502 - accuracy: 0.7089 - val_loss: 2.2393 - val_accuracy: 0.5732
Epoch 5/20
137666/137666 [==============================] - 2057s 15ms/step - loss: 1.0528 - accuracy: 0.7277 - val_loss: 2.3375 - val_accuracy: 0.5647
Epoch 6/20
137666/137666 [==============================] - 2047s 15ms/step - loss: 0.9824 - accuracy: 0.7437 - val_loss: 2.4223 - val_accuracy: 0.5614
Epoch 7/20
137666/137666 [===========

### Test Model

In [36]:
from keras.preprocessing.sequence import pad_sequences

# sample python input
"""
num = 46
print
"""
encoded_text = [2,13,35,5]
seq_len = 6
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre', value = int(vocab['<pad_token>']))
print(encoded_text, pad_encoded[0])

print("Top 3 Suggestions:")
#for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
#    print(idToWord[i])

[2, 13, 35, 5] [1001 1001    2   13   35    5]
Top 3 Suggestions:
